<a href="https://colab.research.google.com/github/Yewon-dev/boostcamp-AI-Tech/blob/master/AI-Paper-Review/Attention_Is_All_You_Need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Paper](https://arxiv.org/pdf/1706.03762.pdf) / 
[Implementation guide](http://nlp.seas.harvard.edu/2018/04/03/attention.html)

**Key Words**
- Encoder-Decoder Stacks
- Attention
- Scaled Dot-Product Attention
- Multi-Head Attention
- Position-wise Feed-Forward Networks
- Embedding and Softmax
- Positional Encoding


# Abstract 

본 논문은 attention mechanisms에 기반을 둔 **Transformer**을 제안하고 있다. 

그동안 RNN, LSTM, GRU과 같은 모델들이 sequence modeling에 효과적으로 사용되었다.

다만 Recurrent model들은 한번에 한 단어씩 넣는 방법처럼 sequence에 들어있는 각각의 token들의 `(1) postion 정보들을 먼저 정렬`한 후, 이것을 `(2) 반복적으로 input`에 넣고 `(3) hidden state를 update`하는 방법으로 진행된다. 이는 병렬적으로 단어를 처리하기 힘들고, 학습되는 문장의 길이가 길어지면 Long-term dependency problem이 생기게 된다.

이를 보안하고자 제안된 **Attention**은 한 번의 행렬곱으로 전체의 sequence에 대한 계산이 가능하다.

-----
# Transformer Architecture

<img src="https://lilianweng.github.io/lil-log/assets/images/transformer.png" width=800>



먼저 개괄적으로 Transformer의 구조를 살펴보면, Encoder-Decoder 형태로 되어 있다는 것을 알 수 있다.



-----
## 1. Encoder-Decoder
**Encoder** 
- Composed of a stack of $N$ identical layers. (문장 속 단어의 개수 = $N$)
- 하나의 Encoder layer는 두 개의 sub-layers로 이루어져 있다. Multi-head **Self-Attention layer**과 Position wise fully connected **Feed-Forward layer**이다.
- 각 sub-layer 이후에 **Layer-Norm**이 residual connection 형태로 수행된다.

**Decoder**
- Decoder는 3개의 sub-layer로 이루어져있다. Encoder와 동일한 2개의 sub-layers (Self-Attention, Feed-Forward)와 Encoder의 Output을 대입하여 attention을 수행하는 1개의 sub-layer (Encoder-Decoder attention)로 구성되어있다.
- self-attention layer는 이전에 나왔던 단어들만 참고할 수 있도록 Mask를 씌운 Masked Multi-Head Attention을 사용한다.




### 1.1 Attention
- Attention은 **하나의 단어와 다른 위치에 있는 단어들의 관계**를 나타내는 layer로 Query, Key, Value 벡터를 이용해서 계산하는 방법이다.

- `Query`는 질문을 하는 주체, `Key`는 attention을 수행하는 대상(단어)이라고 할 수 있다.

Multi-Head Attention의 내부에서는 **Scale Dot-Product Attention**이라는 구조를 사용한다.

### 1.2 Scale Dot-Product Attention

<img src="https://production-media.paperswithcode.com/methods/SCALDE.png" width=300>

1. Query와 Key가 행렬곱을 한다.
2. SoftMax 연산을 위해 Key의 루트값으로 나누는 값을 Scale Layer에 사용한다.
3. Mask 벡터는 Decoder의 Masked Multi-Head Attention에서 사용한다. 이전의 sequence만 사용하기 위해 이후의 sequence들과의 관계 값을 -infinity로 설정하여 softmax의 확률을 0으로 만든다.
4. Softmax를 이용해서 각각의 Key에 대해 Query의 가중치를 확률값으로 구한다.
5. 앞서 구한 확률값을 Value와 행렬곱을 해서 Attention Value 값을 만든다.

Attention의 함수는 다음과 같이 정의된다.
$$Attention(Q,K,V) = softmax( \frac{QK^T}{\sqrt{d_k}} )V$$

### 1.3 Multi-Head Attention

앞서 1.2의 과정들이 각각의 head마다 이루어진 후,
concat으로 합쳐서, 기존의 Embedding 차원과 같은 차원의 output을 만들기 위해 Linear layer을 수행한다.

$$ MultiHead(Q,K,V) = Concat(head_1,...,head_h)W^O$$
$$head_i = Attention(QW^Q_i,KW^K_i,VW^V_i)$$

### 1.4 Positional Encoding

Transformer은 RNN과 다르게 단어의 위치에 대한 정보를 embedding한다. 이것을 **Positional Encoding**이라고 하는데, 본 논문에서는 sin과 cos같은 주기함수를 사용해서 입력을 넣었다.

$$PE_{(pos,2i)} = sin(pos/10000^{2i/d_{model}})$$
$$PE_{(pos,2i+1)} = cos(pos/10000^{2i/d_{model}})$$